## About The Notebook
In this notebook, we will explore a `multi-agent system` designed to detect `user queries related to structured and unstructured data`. The system will intelligently fetch and summarize the requested data from Kaggle datasets, providing a practical exercise for students to understand data handling and analysis using Python.


## 1. Install Necessary Libraries

To begin, we need to install the required libraries. These libraries will help us download datasets, manipulate data, and interact with a SQL database.

In [1]:
# install necessary libraries
!pip install openai pandas sqlalchemy kagglehub


## 2. Import Libraries

Next, we will import the necessary libraries that we just installed. Each library serves a specific purpose in our data processing workflow.

In [2]:
import kagglehub
import os
import pandas as pd
from sqlalchemy import create_engine, text
from openai import OpenAI


## 3. Download Datasets from Kaggle

In this step, we will download both structured and unstructured datasets from Kaggle using the `kagglehub` library. The structured dataset contains tabular data, while the unstructured dataset contains text data.

In [3]:
# Download datasets from Kaggle
structured_path = kagglehub.dataset_download("dillonmyrick/bike-store-sample-database")
unstructured_path = kagglehub.dataset_download("ranand60/unstructured-text-language-data")

print("Structured Dataset Path:", structured_path)
print("Unstructured Dataset Path:", unstructured_path)

100%|██████████| 92.2k/92.2k [00:00<00:00, 92.8MB/s]

Extracting files...


100%|██████████| 416k/416k [00:00<00:00, 84.6MB/s]

Extracting files...
Structured Dataset Path: /root/.cache/kagglehub/datasets/dillonmyrick/bike-store-sample-database/versions/3
Unstructured Dataset Path: /root/.cache/kagglehub/datasets/ranand60/unstructured-text-language-data/versions/1



## 4. Set Up the SQL Database

Now, we will set up a SQLite database to store the structured data. We will create a database file and establish a connection to it.

In [4]:
# Set up the SQL database
db_path = "bike_store_data.db"
engine = create_engine(f'sqlite:///{db_path}')
conn = engine.connect()


### 4.1 Load Structured Data into SQL Database

We will load various CSV files from the structured dataset into the SQL database. Each CSV file corresponds to a different table in the database.

In [5]:
# Load structured data into SQL database
structured_files = {
    "brands": os.path.join(structured_path, "brands.csv"),
    "categories": os.path.join(structured_path, "categories.csv"),
    "customers": os.path.join(structured_path, "customers.csv"),
    "order_items": os.path.join(structured_path, "order_items.csv"),
    "orders": os.path.join(structured_path, "orders.csv"),
    "products": os.path.join(structured_path, "products.csv"),
    "staffs": os.path.join(structured_path, "staffs.csv"),
    "stocks": os.path.join(structured_path, "stocks.csv"),
    "stores": os.path.join(structured_path, "stores.csv"),
}

for table_name, file_path in structured_files.items():
    df = pd.read_csv(file_path)
    df.to_sql(table_name, con=conn, if_exists='replace', index=False)
    print(f"Table '{table_name}' created successfully.")

Table 'brands' created successfully.
Table 'categories' created successfully.
Table 'customers' created successfully.
Table 'order_items' created successfully.
Table 'orders' created successfully.
Table 'products' created successfully.
Table 'staffs' created successfully.
Table 'stocks' created successfully.
Table 'stores' created successfully.



## 5. Load Unstructured Data

Next, we will load the unstructured data from an Excel file. This data will be used for text analysis and summarization.

In [6]:
# Load unstructured data
unstructured_excel = os.path.join(unstructured_path, "Unstructured Data English.xlsx")
unstructured_data = pd.read_excel(unstructured_excel)
print("Loaded unstructured data from:", unstructured_excel)

Loaded unstructured data from: /root/.cache/kagglehub/datasets/ranand60/unstructured-text-language-data/versions/1/Unstructured Data English.xlsx


## 6. Describe your Agents

### AGENT 1: Structured Data Agent
This agent is responsible for `handling queries related to structured data stored in the SQL database`. It processes SQL queries to retrieve data and summarizes the results when applicable.

In [7]:
# Agent 1: Structured Data Agent
def structured_data_agent(user_query):
    structured_data = None
    structured_summary = None

    if "low stock" in user_query.lower():
        query = """
        SELECT p.product_name, st.quantity, s.store_name, s.city
        FROM products p
        JOIN stocks st ON p.product_id = st.product_id
        JOIN stores s ON st.store_id = s.store_id
        WHERE st.quantity < 10
        ORDER BY st.quantity ASC;
        """
        structured_data = pd.read_sql_query(text(query), con=conn)

        # Generate a summary for structured data
        if not structured_data.empty:
            product_names = ", ".join(structured_data['product_name'][:5]) + "..."
            total_low_stock_items = len(structured_data)
            structured_summary = (
                f"There are {total_low_stock_items} products with low stock. "
                f"Some examples include: {product_names}"
            )

    return structured_data, structured_summary

### AGENT 2: Unstructured Data Agent
This `agent handles unstructured textual data`. It searches for relevant information based on keywords and uses OpenAI GPT to summarize matched text, providing insights from unstructured data sources.

In [8]:
# Agent 2: Unstructured Data Agent
def unstructured_data_agent(user_query):
    unstructured_data_summary = None

    if "feedback" in user_query.lower() or "review" in user_query.lower():
        keyword = "Kraft"  # Example keyword for searching unstructured data
        matched_rows = unstructured_data[
            unstructured_data.apply(
                lambda row: keyword.lower() in row.astype(str).str.lower().to_string(),
                axis=1
            )
        ]
        limited_rows = matched_rows.head(10)  # Limit to first 10 rows

        if not limited_rows.empty:
            combined_text = " ".join(limited_rows.astype(str).to_string(index=False))

            # Summarize text using GPT
            client = OpenAI(api_key = "Your-api-key")
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that summarizes texts."},
                    {"role": "user", "content": f"Summarize the following text:\n\n{combined_text}"}
                ],
                max_tokens=150
            )
            unstructured_data_summary = response.choices[0].message.content

    return unstructured_data_summary


## 7. Unified Query Handling Logic

This function will intelligently detect whether the query pertains to structured or unstructured data and fetch the relevant information accordingly.

In [ ]:
# Unified function to handle user queries
def handle_query(user_query):
    structured_data, structured_summary = structured_data_agent(user_query)
    unstructured_data_summary = unstructured_data_agent(user_query)

    return structured_data, structured_summary, unstructured_data_summary


## 8. Test with a User Query

We will simulate a user query to test our multi-agent system. This will help us verify that the system can accurately detect and respond to requests for both structured and unstructured data.

In [ ]:
# Test with a user query
user_query = "Can you identify low stock products and summarize customer feedback about them?"
structured_data, structured_summary, unstructured_data_summary = handle_query(user_query)


## 9. Display Results

Finally, we will display the results of the structured data query and any summaries generated from the unstructured data. This will provide the user with a comprehensive overview based on their query.

In [ ]:
# Step 5: Display results
if structured_data is not None:
    print("\nStructured Data (SQL Query Result):")
    print(structured_data)

if structured_summary is not None:
    print("\nStructured Data Summary:")
    print(structured_summary)

if unstructured_data_summary is not None:
    print("\nUnstructured Data Summary:")
    print(unstructured_data_summary)

## Try the Queries given below for Your SQL Dataset
We used the Low stock Product example. Go ahead and try the rest. Make sure you change the **`Unified Query Handling Logic`** and the **`User Query`**.

In [ ]:
# Query 1: Fetch customer orders with total amount spent
query_1 = """
SELECT c.first_name || ' ' || c.last_name AS customer_name,
       o.order_date,
       SUM(oi.quantity * (oi.list_price - oi.discount)) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_id, o.order_id
ORDER BY total_spent DESC;
"""

# Query 2: Products with low stock
query_2 = """
SELECT p.product_name, s.store_name, st.quantity
FROM products p
JOIN stocks st ON p.product_id = st.product_id
JOIN stores s ON st.store_id = s.store_id
WHERE st.quantity < 10
ORDER BY st.quantity ASC;
"""

query_3 = """
SELECT st.first_name || ' ' || st.last_name AS staff_name,
       SUM(oi.quantity * (oi.list_price - oi.discount)) AS total_sales
FROM staffs st
JOIN orders o ON st.staff_id = o.staff_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY st.staff_id
ORDER BY total_sales DESC;
"""